<!-- # Copyright (c) 2024 Graphcore Ltd. All rights reserved. -->

# Collect value statistics for formats

This notebook computes various statistics for a variety of float formats,
by exhaustively enumerating the values.  Naturally, most of these statistics can be computed directly, and indeed many are already supplied on the `FormatInfo` class as methods, for example `max`, `smallest_subnormal`, etc.  However this method serves as a useful cross-check against the direct formulae.

## Statistics collected

 - name: Format
 - B: Bits in the format
 - P: Precision in bits
 - E: Exponent field width in bits
 - T: Trailing significand field width in bits
 - lt1: Number of values x such that `0 < x < 1`
 - gt1: Number of values x such that `1 < x < Inf`
 - rt16: True if all values are exactly representable in IEEE binary16
 - maxFinite: Largest finite value
 - minFinite: Smallest finite value
 - maxNormal: Largest finite normal value, NaN if all finite values are subnormal
 - minNormal: Smallest positive normal value, NaN if all finite values are subnormal
 - minSubnormal: Smallest positive subnormal value, NaN if no finite values are  - subnormal
 - maxSubnormal: Largest subnormal value, NaN if no finite values are subnormal


In [1]:
%run utils.py
D = pandas_render  # from utils
import pandas
from gfloat import *
from gfloat.formats import *

import numpy as np


def collect_stats(fi: FormatInfo):
    # Generate all values
    values = [decode_float(fi, i) for i in range(2**fi.bits)]
    df = pandas.DataFrame(values)

    # Extract format information parameters
    E = fi.expBits
    S = fi.tSignificandBits

    # Compute statistics: lt1,gt1
    fval = df["fval"]
    total_01 = fval.between(0, 1, inclusive="neither").sum()
    total_1Inf = fval.between(1, np.inf, inclusive="neither").sum()

    # Compute statistics: maxFinite,minFinite
    finite_vals = fval[np.isfinite(fval)]
    maxFinite = finite_vals.loc[finite_vals.idxmax()]
    minFinite = finite_vals.loc[finite_vals.idxmin()]

    # Compute statistics: maxNormal,minNormal
    normal_vals = fval[(df["fclass"] == FloatClass.NORMAL) & (fval > 0)]
    maxNormal = normal_vals.loc[normal_vals.idxmax()] if normal_vals.any() else np.nan
    minNormal = normal_vals.loc[normal_vals.idxmin()] if normal_vals.any() else np.nan

    # Compute statistics: minSubnormal
    pos_subnormal = fval[(df["fclass"] == FloatClass.SUBNORMAL) & (fval > 0)]
    maxSubnormal = (
        pos_subnormal.loc[pos_subnormal.idxmax()] if pos_subnormal.any() else np.nan
    )
    minSubnormal = (
        pos_subnormal.loc[pos_subnormal.idxmin()] if pos_subnormal.any() else np.nan
    )

    # Compute roundtrips: rt16, rt32
    with np.errstate(over="ignore"):
        rt16 = (np.float64(np.float16(fval)) == np.float64(fval)) | ~np.isfinite(fval)
        rt32 = (np.float64(np.float32(fval)) == np.float64(fval)) | ~np.isfinite(fval)

    rt16 = rt16.all()
    rt32 = rt32.all()
    assert rt32  # If not, we should include rt32 in the table

    # Assemble tuple
    return dict(
        name=fi.name,
        B=fi.bits,
        P=fi.precision,
        E=E,
        T=S,
        lt1=total_01,
        gt1=total_1Inf,
        rt16=rt16,
        maxFinite=maxFinite,
        minFinite=minFinite,
        maxNormal=maxNormal,
        minNormal=minNormal,
        minSubnormal=minSubnormal,
        maxSubnormal=maxSubnormal,
    )


stats = [collect_stats(fi) for fi in all_formats if fi.bits <= 16]
df = pandas.DataFrame(stats)
D(df)

name,B,P,E,T,lt1,gt1,rt16,maxFinite,minFinite,maxNormal,minNormal,minSubnormal,maxSubnormal
ocp_e8m0,8,1,8,0,127,127,False,170141183460469231731687303715884105728.000000,0.000000,170141183460469231731687303715884105728.000000,0.000000,nan,nan
ocp_int8,8,8,0,7,63,63,True,1.984375,-2.000000,nan,nan,0.015625,1.984375
ocp_e2m1,4,2,2,1,1,5,True,6.000000,-6.000000,6.000000,1.000000,0.500000,0.500000
ocp_e2m3,6,4,2,3,7,23,True,7.500000,-7.500000,7.500000,1.000000,0.125000,0.875000
ocp_e3m2,6,3,3,2,11,19,True,28.000000,-28.000000,28.000000,0.250000,0.062500,0.187500
ocp_e4m3,8,4,4,3,55,70,True,448.000000,-448.000000,448.000000,0.015625,0.001953,0.013672
ocp_e5m2,8,3,5,2,59,63,True,57344.000000,-57344.000000,57344.000000,0.000061,0.000015,0.000046
p3109_p1,8,1,7,0,62,63,False,9223372036854775808.000000,-9223372036854775808.000000,9223372036854775808.000000,0.000000,nan,nan
p3109_p2,8,2,6,1,63,62,False,2147483648.000000,-2147483648.000000,2147483648.000000,0.000000,0.000000,0.000000
p3109_p3,8,3,5,2,63,62,True,49152.000000,-49152.000000,49152.000000,0.000031,0.000008,0.000023


## Emit HTML table

In [2]:
# Special rendering for float values - if they don't render nicely in 10.5g,
# use float_pow2str
def render_float(v):
    s = f"{v:8.5g}"
    if not "e" in s and float(s) == v:
        return s
    else:
        return float_pow2str(v)


for field in (
    "maxFinite",
    "minFinite",
    "maxNormal",
    "minNormal",
    "minSubnormal",
    "maxSubnormal",
):
    df[field] = df[field].map(render_float)

D(df)

name,B,P,E,T,lt1,gt1,rt16,maxFinite,minFinite,maxNormal,minNormal,minSubnormal,maxSubnormal
ocp_e8m0,8,1,8,0,127,127,False,1*2^127,1*2^-127,1*2^127,1*2^-127,nan,nan
ocp_int8,8,8,0,7,63,63,True,127/64*2^0,-2,nan,nan,0.015625,127/64*2^0
ocp_e2m1,4,2,2,1,1,5,True,6,-6,6,1,0.5,0.5
ocp_e2m3,6,4,2,3,7,23,True,7.5,-7.5,7.5,1,0.125,0.875
ocp_e3m2,6,3,3,2,11,19,True,28,-28,28,0.25,0.0625,0.1875
ocp_e4m3,8,4,4,3,55,70,True,448,-448,448,0.015625,1*2^-9,7/4*2^-7
ocp_e5m2,8,3,5,2,59,63,True,57344,-57344,57344,1*2^-14,1*2^-16,3/2*2^-15
p3109_p1,8,1,7,0,62,63,False,1*2^63,-1*2^63,1*2^63,1*2^-62,nan,nan
p3109_p2,8,2,6,1,63,62,False,1*2^31,-1*2^31,1*2^31,1*2^-31,1*2^-32,1*2^-32
p3109_p3,8,3,5,2,63,62,True,49152,-49152,49152,1*2^-15,1*2^-17,3/2*2^-16
